In [1]:
import glob, sys

from omegaconf import OmegaConf
from tqdm.auto import tqdm
from grams.prelude import GRAMS, ROOT_DIR, DATA_DIR, I, WikidataSemanticModelHelper
from sm.prelude import M, O

In [2]:
cfg = OmegaConf.load(ROOT_DIR / "grams.yaml")
grams = GRAMS(DATA_DIR, cfg)

In [3]:
cwd = ROOT_DIR / "examples/t2dv2"
tables = [I.LinkedTable.from_csv_file(infile) for infile in glob.glob(str(cwd / "tables/*.csv")) if infile.find("table_linker") == -1]

In [4]:
annotations = [grams.annotate(table, verbose=True) for table in tqdm(tables)]

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
batch_annotator.batch_annotate([
    (table.get_friendly_fs_id(), "", table)
    for table in tables
], start_index=0)

In [13]:
!ls "{cwd}/ground-truth"

29414811_2_4773219892816395776.json
